# Preprocess the dropseq and smFISH data

Data: https://drive.google.com/drive/folders/1uos3DDymyrh3ZyxveDhqQIcOyD1brxeL

In [2]:
import pandas as pd
import seaborn as sns
import scanpy as sc
import scipy.sparse as sparse
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as opt
import itertools
import scanpy as sc

### Process DropSeq data

In [3]:
data_path = '/data_volume/memento/saver/'

partial = pd.read_csv('/data_volume/memento/saver/melanoma_dropseq.csv', index_col=0, sep=',').T
dropseq = pd.read_csv('GSE99330_dropseqRPM.txt', sep=' ').T.loc[partial.index]/1e6

# Reverse calculate cell sizes
high_expr = partial.mean(axis=0).sort_values().tail(20).index.tolist()
dropseq_sf = (partial[high_expr]/dropseq[high_expr]).mean(axis=1)
dropseq = dropseq*dropseq_sf.values.reshape(-1,1)

# Create sparse matrix and save
X = sparse.csr_matrix(dropseq.values)
dropseq_adata = sc.AnnData(X=X, obs=pd.DataFrame(index=dropseq.index), var=pd.DataFrame(index=dropseq.columns))

dropseq_adata.write(data_path + 'full_dropseq.h5ad')

### Compute number of overlap genes

In [4]:
smfish = pd.read_csv('/data_volume/memento/saver/fishSubset.txt', index_col=0, sep=' ')
# smfish = smfish[smfish['GAPDH'] > 0]
fish_gapdh_sf = (smfish['GAPDH']+1).values
# smfish_normalized = smfish#/fish_gapdh_sf.reshape(-1, 1)
# smfish = pd.read_csv('fishSubset (1).txt', sep=' ', index_col=0)

overlap_genes = list(set(dropseq.columns) & set(smfish.columns))
overlap_genes = dropseq.mean(axis=0)[overlap_genes][dropseq.mean(axis=0)[overlap_genes] > 0].index.tolist()

print(len(overlap_genes))

# smfish = smfish[overlap_genes].fillna(0.0)

26
